In [1]:
# Paso 1: Instalar Dependencias
%pip install backtesting pandas numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 3.8 MB/s eta 0:00:0000:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 58.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.9/83.9 kB 2.6 MB/s eta 0:00:00
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173916 sha256=a3ee44b36d7416a6c38a2ca06f8f42ff971413c38dacc4f8d56889053d113f70
  Stored in directory: /home/codespace/.cache/pip/wheels/e2/30/7f/19cbe31987c6ebdb47f1f510343249066711609e3da2d57176
Successfully built backtesting
Note: you may need to restart the kernel to use updated packages.


In [1]:
# Paso 2: Importar las Librerías Necesarias
import pandas as pd
import numpy as np
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA, GOOG

/home/codespace/.python/current/lib/python3.10/site-packages/backtesting/test/__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),
/home/codespace/.python/current/lib/python3.10/site-packages/backtesting/test/__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),


In [2]:
# Paso 3: Definir la Estrategia de Mean Reversion
class MeanReversionStrategy(Strategy):
    def init(self):
        # Inicializa las Bandas de Bollinger
        price = self.data.Close
        self.ma = self.I(SMA, price, 20)  # Media Móvil Simple de 20 períodos
        self.upper_band = self.ma + 2 * price.rolling(20).std()
        self.lower_band = self.ma - 2 * price.rolling(20).std()

    def next(self):
        price = self.data.Close[-1]

        # Señal de compra: el precio cruza por debajo de la banda inferior
        if crossover(self.lower_band, price):
            self.buy()

        # Señal de venta: el precio cruza por encima de la banda superior
        elif crossover(price, self.upper_band):
            self.sell()


In [7]:
# importar data 

datapath = '../backtesting/data/AAPL.csv'
df = pd.read_csv(datapath, parse_dates=True, index_col=0, )
# data = (df / 1e6).assign(Volume=df.Volume * 1e6)  # μBTC OHLC prices


In [8]:
# df = pd.DataFrame(GOOG)
# df.rename(columns={"Open": "open", "High": "high", "Low": "low", "Close": "close", "Volume": "volume"}, inplace=True)
df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2012-01-03,14.621429,14.732143,14.607143,14.686786,12.433824,302220800
2012-01-04,14.642857,14.810000,14.617143,14.765714,12.500646,260022000
2012-01-05,14.819643,14.948214,14.738214,14.929643,12.639424,271269600
2012-01-06,14.991786,15.098214,14.972143,15.085714,12.771558,318292800
2012-01-09,15.196429,15.276786,15.048214,15.061786,12.751302,394024400


In [9]:
bt = Backtest(df, MeanReversionStrategy, cash=10000, commission=.002)
stats = bt.run()
print(stats)
bt.plot()

AttributeError: '_Array' object has no attribute 'rolling'